# Solution

This notebook shows the procedure we used to generate the final submission for the challenge. We first used the median of 146 models (selected by leaving 1 drug out for NK cells) as the base prediction. This model was trained only on the top 64 most variable genes (sorted by variance), and was used to predict the ~18K signed log-pvalues.

We then trained the same NN on a subset of the data. This produced again 146 models that we used to generate a median prediction only on that subset of the data. 

Finally, we replaced the base predictions with the for the final submission.

In [ ]:
import scape
import pandas as pd
import numpy as np

scape.__version__, pd.__version__, np.__version__

In [ ]:
df_de = scape.io.load_slogpvals("_data/de_train.parquet")
df_lfc = scape.io.load_lfc("_data/lfc_train.parquet")

# Make sure rows/columns are in the same order
df_lfc = df_lfc.loc[df_de.index, df_de.columns]
df_de.shape, df_lfc.shape

In [ ]:
# We select only a subset of the genes for the model (top most variant genes)
n_genes = 64
top_genes = scape.util.select_top_variable([df_de], k=n_genes)
#scm = scape.model.create_default_model(n_genes, df_de, df_lfc)
#scape.util.plot(scm.model, show_shapes=True)

## Base predictions

In [ ]:
cell = "NK cells"
drugs = df_de.loc[df_de.index.get_level_values("cell_type") == cell].index.get_level_values("sm_name").unique().tolist()
len(drugs)

In [ ]:
df_id_map = pd.read_csv("_data/id_map.zip")
df_sub = pd.read_csv("_data/sample_submission.zip", index_col = 0)
df_sub_ix = df_id_map.set_index(["cell_type", "sm_name"])
df_sub_ix

In [ ]:
base_predictions = []
for i, d in enumerate(drugs):
    print(i, d)
    scm = scape.model.create_default_model(n_genes, df_de, df_lfc)
    result = scm.train(
        val_cells=[cell], 
        val_drugs=[d],
        input_columns=top_genes,
        epochs=300,
        output_folder="_models",
        config_file_name="config.pkl",
        model_file_name=f"drug{i}.keras",
        baselines=["zero", "slogpval_drug"],
    )
    # Collect prediction in the OOF data
    df_pred = scm.predict(df_sub_ix)
    df_pred = df_pred.loc[:, df_sub.columns]
    base_predictions.append(df_pred)

df_sub = pd.DataFrame(np.median(base_predictions, axis=0), index=df_sub_ix.index, columns=df_sub.columns)
df_sub


## Enhanced predictions

We selected a subset of the dataset consisting of the top 256 genes and top 60 drugs (sorted by variance). We trained the same model as before on this subset of the data, and used the 146 models to generate a median prediction on this subset of the data. We finally merged the results with the base predictions using a weighted mean (0.80 for the enhaced predictions in the subset of 256 genes and 60 drugs, 0.20 for the base prediction)

In [ ]:
sub_drugs = df_sub_ix.index.get_level_values("sm_name").unique().tolist()
len(sub_drugs)

In [ ]:
min_n_top_drugs = 50
n_genes = 256

# This time, exclude control drugs for the calculation of the top genes, in order to
# introduce more variability in the model
top_genes = scape.util.select_top_variable([df_de], k=n_genes, exclude_controls=True)

In [ ]:
df_drug_effects = pd.DataFrame(df_de.T.pow(2).mean().pow(0.5).groupby("sm_name").mean().sort_values(ascending=False), columns=["effect"])
df_drug_effects["effect_norm"] = (df_drug_effects["effect"] / df_drug_effects["effect"].sum())*100
df_drug_effects

In [ ]:
top_sub_drugs = df_drug_effects.loc[sub_drugs].sort_values("effect", ascending=False).head(min_n_top_drugs).index.tolist()
len(top_sub_drugs)

In [ ]:
top_all_drugs = df_drug_effects.head(min_n_top_drugs).index.tolist()
top_drugs = set(top_all_drugs) | set(top_sub_drugs)
len(top_drugs)

In [ ]:
df_de_c = df_de[df_de.index.get_level_values("sm_name").isin(top_drugs)]
df_de_c = df_de_c.loc[:, top_genes]
df_de_c

In [ ]:
df_lfc_c = df_lfc.loc[df_de_c.index, df_de_c.columns]
df_lfc_c.shape

In [ ]:
enhanced_predictions = []
for i, d in enumerate(top_drugs):
    print(i, d)
    scm = scape.model.create_default_model(n_genes, df_de_c, df_lfc_c)
    result = scm.train(
        val_cells=[cell], 
        val_drugs=[d],
        input_columns=top_genes,
        epochs=800,
        output_folder="_models",
        config_file_name="enhanced_config.pkl",
        model_file_name=f"enhanced_drug{i}.keras",
        baselines=["zero", "slogpval_drug"],
    )
    # Collect prediction in the OOF data
    df_pred = scm.predict(df_sub_ix)
    enhanced_predictions.append(df_pred)

df_sub_enhanced = pd.DataFrame(np.median(enhanced_predictions, axis=0), index=df_sub_ix.index, columns=df_de_c.columns)
df_sub_enhanced


In [ ]:
df_focus = df_sub.copy()
df_focus.update(df_sub_enhanced)
df_focus

In [ ]:
df_submission = 0.80 * df_focus + 0.20 * df_sub
df_submission